In [1]:
import pandas as pd

In [2]:
path = "../Data/Online_Data/trainning_data/corporate_attributes/"
outpath = "../Processed_Data/"

In [3]:
tax_company = pd.read_csv(path+'tax_company.csv')
tax_company.head()

<ipython-input-3-7a46edec183e>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  tax_company = pd.read_excel(path+'tax_company.xlsx')


,entid,region_id,entertype_tax,industry_tax,registertype,economic_type,incometax_rate,collection_type,EMPNUM,year
0,122697049,1,NaN,NaN,15,168,NaN,2,181292.0,2016.0
1,122697049,1,NaN,NaN,9,167,NaN,2,153.0,2015.0
2,122697049,1,NaN,NaN,15,168,0.25,2,185818.0,2017.0
3,959830672,3,0.0,2.0,19,42,0.25,0,13.0,NaN
4,959830672,3,NaN,NaN,19,206,NaN,2,2.0,2016.0


In [18]:
# only keep the latest data
tax_company_latest = tax_company[~tax_company['year'].isna()]
tax_company_latest = tax_company_latest.sort_values(by = ['entid', 'year'])

In [19]:
tax_company_latest.head()

,entid,region_id,entertype_tax,industry_tax,registertype,economic_type,incometax_rate,collection_type,EMPNUM,year
7636,109606,647,NaN,NaN,9,991,NaN,2,24.0,2015.0
7635,109606,647,NaN,NaN,9,991,NaN,2,24.0,2016.0
7637,109606,647,NaN,NaN,9,991,0.25,2,35.0,2017.0
17174,243165,1819,NaN,NaN,9,1244,NaN,2,17.0,2015.0
17173,243165,1822,NaN,NaN,9,1244,NaN,2,14.0,2016.0


In [20]:
tax_company_latest['EMPNUM_Diff'] = tax_company_latest['EMPNUM'].diff()

In [21]:
tax_company_latest['entid_Diff'] = tax_company_latest['entid'].diff()

In [22]:
tax_company_latest.head()

,entid,region_id,entertype_tax,industry_tax,registertype,economic_type,incometax_rate,collection_type,EMPNUM,year,EMPNUM_Diff,entid_Diff
7636,109606,647,NaN,NaN,9,991,NaN,2,24.0,2015.0,NaN,NaN
7635,109606,647,NaN,NaN,9,991,NaN,2,24.0,2016.0,0.0,0.0
7637,109606,647,NaN,NaN,9,991,0.25,2,35.0,2017.0,11.0,0.0
17174,243165,1819,NaN,NaN,9,1244,NaN,2,17.0,2015.0,-18.0,133559.0
17173,243165,1822,NaN,NaN,9,1244,NaN,2,14.0,2016.0,-3.0,0.0


In [23]:
tax_company_latest['EMPNUM_Diff'] = tax_company_latest.apply(lambda row: row.EMPNUM_Diff if row.entid_Diff==0 else 0, axis = 1)

In [24]:
tax_company_latest.columns

Index(['entid', 'region_id', 'entertype_tax', 'industry_tax', 'registertype',
       'economic_type', 'incometax_rate', 'collection_type', 'EMPNUM', 'year',
       'EMPNUM_Diff', 'entid_Diff'],
      dtype='object')

In [25]:
numerical_columns = ['EMPNUM', 'EMPNUM_Diff']

In [26]:
def get_numerical_df(df, columns):
    keep_columns = columns + ['entid']
    df_numerical = df[keep_columns].groupby("entid")
    df_numerical = df_numerical.mean()
    df_numerical = df_numerical.reset_index()
    return df_numerical

In [27]:
df_numerical = get_numerical_df(tax_company_latest, numerical_columns)
df_numerical.shape

(7986, 3)

In [28]:
df_numerical.head()

,entid,EMPNUM,EMPNUM_Diff
0,109606,27.666667,3.666667
1,243165,15.000000,-1.000000
2,285412,2.000000,0.000000
3,439534,5.666667,-2.333333
4,530656,2.000000,-1.000000


In [29]:
categorical_columns = ['registertype', 'collection_type']

In [30]:
def get_categorical_df(df, columns):
    keep_columns = columns + ['entid']
    df_categorical = df[keep_columns].drop_duplicates(subset = 'entid', keep = 'last')
    df_categorical = pd.get_dummies(df_categorical, columns = categorical_columns, drop_first=True)
    return df_categorical

In [31]:
df_categorical = get_categorical_df(tax_company_latest, categorical_columns)

In [32]:
df_categorical.head()

,entid,registertype_2,registertype_3,registertype_4,registertype_5,registertype_6,registertype_8,registertype_9,registertype_10,registertype_11,...,registertype_26,registertype_27,registertype_28,registertype_29,registertype_30,registertype_31,registertype_33,registertype_35,registertype_37,registertype_38
7637,109606,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17175,243165,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23986,285412,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,439534,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11950,530656,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_categorical.shape

(7986, 34)

In [34]:
df_new = pd.merge(df_numerical, df_categorical, on = "entid")
df_new.head()

,entid,EMPNUM,EMPNUM_Diff,registertype_2,registertype_3,registertype_4,registertype_5,registertype_6,registertype_8,registertype_9,...,registertype_26,registertype_27,registertype_28,registertype_29,registertype_30,registertype_31,registertype_33,registertype_35,registertype_37,registertype_38
0,109606,27.666667,3.666667,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,243165,15.000000,-1.000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,285412,2.000000,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,439534,5.666667,-2.333333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,530656,2.000000,-1.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# add on the rest categorical data
selected_latest = tax_company_latest[['entid', 'region_id', 'economic_type']].drop_duplicates(subset = 'entid', keep = 'last')
selected_latest.head()

,entid,region_id,economic_type
7637,109606,647,991
17175,243165,1822,1244
23986,285412,1499,1059
7499,439534,657,1203
11950,530656,893,1164


In [36]:
df_new = pd.merge(selected_latest, df_new, on = "entid")
df_new.head()

,entid,region_id,economic_type,EMPNUM,EMPNUM_Diff,registertype_2,registertype_3,registertype_4,registertype_5,registertype_6,...,registertype_26,registertype_27,registertype_28,registertype_29,registertype_30,registertype_31,registertype_33,registertype_35,registertype_37,registertype_38
0,109606,647,991,27.666667,3.666667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,243165,1822,1244,15.000000,-1.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,285412,1499,1059,2.000000,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,439534,657,1203,5.666667,-2.333333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,530656,893,1164,2.000000,-1.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df_new.shape

(7986, 38)

In [38]:
df_new = df_new.dropna()
df_new.shape

(7976, 38)

In [39]:
df_new.to_csv(outpath+'tax_company_processed.csv', index = 0)